# **RECONHECIMENTO DE TEXTOS GERADOS POR INTELIGÊNCIA ARTIFICIAL: IMPLEMENTAÇÃO DAS PROPOSTAS APRESENTADAS**

---

SCC0633/SCC5908 - Processamento de Linguagem Natural - 2024

Professor: Thiago A.S Pardo.
Estagiário PAE: Germano A. Z. Jorge

## **Grupo: Generar**

* Jônatas Alves Lopes (11796552)
* Julio Cesar Galdino (15112607)
* Maria Victória Brandão Barros (12608692)
* Rebeca Vieira Carvalho (12543530)
* Thierry de Souza Araújo (12681094)

## **Inicialização do ambiente**

Para que a execução do sistema e de todas as suas partes, é necessário realizar as seguintes instalações e importações.  

### **Instalando bibliotecas adicionais**

In [ ]:
pip install tokenizer

In [ ]:
pip install pyspellchecker

### **Importações**

In [ ]:
import pandas as pd
from tokenizer import tokenize
import nltk
from collections import Counter
from spellchecker import SpellChecker

### **NLTK downloads**
Esses arquivos são importantes para o funcionamento da biblioteca NLTK e suas funções de tageamento.

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### **Importação do Dataset**

#### Com arquivo baixado

In [ ]:
import csv
df = pd.read_csv('AI_Human.csv')

#### Pegando dataframe

In [ ]:
df

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
...,...,...
487230,Tie Face on Mars is really just a big misunder...,0.0
487231,The whole purpose of democracy is to create a ...,0.0
487232,I firmly believe that governments worldwide sh...,1.0
487233,I DFN't agree with this decision because a LFT...,0.0


### **Separando os samples**

In [ ]:
# Get n texts from each class
n = 2000
m = 1000
df_ai = df[df['generated'] == 1].sample(n)
test_ai=df[df['generated'] == 1].drop(df_ai.index).sample(m)

df_not_ai = df[df['generated'] == 0].sample(n)
test_not_ai=df[df['generated'] == 0].drop(df_not_ai.index).sample(m)

# Get all texts in order: AI and Human
concat_df = pd.concat([df_ai, df_not_ai])
concat_df

,text,generated
312273,"In Che United Spaces, car ownership is a near ...",1.0
343807,There is no single answer to whether young peo...,1.0
473629,"In this modern society, it'seems that every li...",1.0
209714,When people are not accepted For who they trul...,1.0
331884,The told is currently in a state of flux. Clim...,1.0
...,...,...
165486,Have you known that being honest is sometimes ...,0.0
339415,The process OO closing a president should be c...,0.0
297379,Driving using a phone?\n\nWhen you are driving...,0.0
262527,How should a summer project be designed to ass...,0.0


## **Implementação**

### **Funções Utils**
Essas são as funções que serão utilizadas ao longo do código por várias etapas, dessa forma estão unidas nessa seção.


In [ ]:
# This function tokenize a text removing blank spaces
def tokenize_phrase(text):
  words = tokenize(text)
  words_txt = [word.txt for word in words if word.txt != '']
  return words_txt

In [ ]:
# Calculates mean square erros of a dataframe, returns the percentage of each classification
def freq_mean_square(freq, freq_ai, freq_not_ai):
  sum_ai = 0
  sum_not_ai = 0
  n = len(freq.items())

  for key, value in freq.items():
    if key in freq_ai:
      sum_ai += (value - freq_ai[key])**2

  for key, value in freq.items():
    if key in freq_not_ai:
      sum_not_ai += (value - freq_not_ai[key])**2
  n = sum_ai + sum_not_ai

  # Closer to AI x Closer to Human
  return 1-sum_ai/n, 1-sum_not_ai/n

In [ ]:
# Calculates mean square erros of a list, returns the percentage of each classification
def freq_mean_square_list(freq, freq_ai, freq_not_ai):
  sum_ai = 0
  sum_not_ai = 0
  n = len(freq)

  for i, value in enumerate(freq):
    sum_ai += (value - freq_ai[i])**2
  for i, value in enumerate(freq):
    sum_not_ai += (value - freq_not_ai[i])**2

  n = sum_ai + sum_not_ai

  # Closer to AI x Closer to Human
  return [1-sum_ai/n, 1-sum_not_ai/n]

# **Critérios de Classificação**
Aqui estão descritas todas a implementação dos critérios utilizados para a realização da classificação. Existem no total quatro critérios, sendo eles:

1.   Repetição de tags
2.   Uso de sinais de pontuação
3.   Comprimento das palavras
4.   Desvios gramaticais
5.   Junção de todos os critérios


## **1. Repetição de tags**


Nesse critério é levado em consideração a frequência das classes gramaticais de um texto, ou seja, quantas vezes é repetido os adjetivos, nomes, advérbios, etc. Quando é levado em consideração os dois tipos de textos, espera-se que aqueles que foram feitos por AI tenham valores específicos e similares, assim como os feitos por humanos. É importante ressaltar que devido a possíveis erros no dataset e nos textos, os valores podem não ser classificados corretamente.

<br>

**Critérios utilizados**
Para realizar a comparação o sistema realiza os seguintes passos:

1.   Tokenização e tageamento
2.   Cálculo de frequência de cada tag, as k maiores frequências são armazenadas.

  **Exemplo:** <br>
  Se o texto em questão for: *'I am from Canada, my name is Paul Canada'*

  O resultado do tagueamento seria:

  [('I', 'PRP'),
 ('am', 'VBP'),
 ('from', 'IN'),
 ('Canada', 'NNP'),
 (',', ','),
 ('my', 'PRP$'),
 ('name', 'NN'),
 ('is', 'VBZ'),
 ('Paul', 'NNP'),
 ('Canada', 'NNP')]

 Legenda:
 * 'Adjective': ['JJ']
 * 'Noun': ['NN', 'NNP', 'NNPS']
 * 'Adverb': ['RB', 'RBR', 'RBS']
 * 'Verb': ['VBD', 'VBZ', 'VBG', 'VBP', 'VB']

 Nesse caso, portanto, se k=2, o texto teria a seguinte frequência

 * Adjectives: 0, 0
 * Noun: 2(Canada), 1(name)
 * Adverbs: 0, 0
 * Verbs: 1(am), 1(is)

3.   Comparação dos resultados

  O processo é realizado em todos os valores do sample de IA e Humano para adquirir a média das frequências de cada classe. Após isso o texto a ser classificado é comparado com os resultados e a função de Erro Quadrático Médio é aplicada.Os valores resultantes são a porcentagem de semelhança entre as classes: AI e Não AI.



In [ ]:
# Description: Makes the parsing(tagging) of a certain text
# Params: text to be tagged
# Return: the text tokenized and tagged
def words_by_tag(text):
  wordtokens = nltk.word_tokenize(text)
  tagged_words = nltk.pos_tag(wordtokens)
  return tagged_words

text = 'I am from Canada, my name is Paul Canada'
words_by_tag(text)

[('I', 'PRP'),
 ('am', 'VBP'),
 ('from', 'IN'),
 ('Canada', 'NNP'),
 (',', ','),
 ('my', 'PRP$'),
 ('name', 'NN'),
 ('is', 'VBZ'),
 ('Paul', 'NNP'),
 ('Canada', 'NNP')]

In [ ]:
# Description: Apply tagging in all texts of a dataframe
# Params: dataframe with texts
# Return: dictionary with all texts tagged
def get_tagged_texts(_df):
  # Create an empty dictionary to store tagging for each text
  tagged_texts = {}

  # Iterate over each row in the DataFrame
  for index, row in _df.iterrows():
    text_id = index  # Assuming index represents text IDs
    text = row['text']

    # Tagging
    tagged_text = words_by_tag(text)

    # Store the tagged texts in the dictionary
    tagged_texts[text_id] = tagged_text

  return tagged_texts

In [ ]:
# Description: Returns k word mean frequencies
# Params: dictionary of Counters(frequencies), value k
# Return: dictionary with the highest k frequencies of k words
def mean_word_frequency(frequencies_dict, k=5):

  sum_freq = [0]*k
  n = len(frequencies_dict.items())

  for key, value in frequencies_dict.items():
    # sort frequencies of words
    sorted_freq = sorted(dict(value).values(), reverse=True)[:k]
    # Equalize list size with zeros
    sorted_freq += [0]*(k-len(sorted_freq))
    # sum k frequencies of all texts from frequencies_list
    sum_freq = [x + y for x, y in zip(sum_freq, sorted_freq)]

  mean_k_freq = [x/n for x in sum_freq]
  return mean_k_freq

In [ ]:
# Description: Get the frequencies only of the specified tags
# Params: dictionary with tagged texts, dictionary of tags to be considered
# Return: dictionary of Counters with all the frequencies of each word
def frequency_by_tag(texts, tags):
  new_dict = {}
  for key, value in texts.items():
    # select only the words in the tags list
    new_list = [word[0] for word in value if word[1] in tags]
    # Counter count the frequency of each word in the list
    new_dict[key] = Counter(new_list)

  return new_dict

In [ ]:
# All tags considered
tags = {'Adjective': ['JJ'], 'Noun': ['NN', 'NNP', 'NNPS'], 'Adverb': ['RB', 'RBR', 'RBS'], 'Verb': ['VBD','VBZ', 'VBG', 'VBP', 'VB']}

In [ ]:
# Description: Get the highest k frequencies by word for each dataframe
# Params: dictionary with AI texts, dictionary with Human texts
# Return: tuple of the highest k frequencies by word for each dataframe
def get_frequencies_by_class(df_ai, df_not_ai):
  tagged_ai = get_tagged_texts(df_ai)
  tagged_not_ai = get_tagged_texts(df_not_ai)

  frequency_tags_ai = {}
  frequency_tags_not_ai = {}

  for key, value in tags.items():

    # AI
    frequencies_ai = frequency_by_tag(tagged_ai, value)
    frequencies_mean_ai = mean_word_frequency(frequencies_ai)

    # Store results to AI
    frequency_tags_ai[key] = frequencies_mean_ai

    # NOT AI
    frequencies_not_ai = frequency_by_tag(tagged_not_ai, value)
    frequencies_mean_not_ai = mean_word_frequency(frequencies_not_ai)

    # Store results to Not AI
    frequency_tags_not_ai[key] = frequencies_mean_not_ai

  return frequency_tags_ai, frequency_tags_not_ai

In [ ]:
frequency_tags_ai, frequency_tags_not_ai = get_frequencies_by_class(df_ai, df_not_ai)

In [ ]:
def classify_by_tag_frequency(text):

  # frequency_tags_ai, frequency_tags_not_ai = get_frequencies_by_class(df_ai, df_not_ai)

  # Inicialization
  tagged_text = { 'text': words_by_tag(text) }
  m_square = []
  sum_ai = 0
  sum_not_ai = 0

  # iterate over the tags
  for key, value in tags.items():

    # Calculate frequency
    frequency_text = frequency_by_tag(tagged_text, value)
    frequency_text_ = mean_word_frequency(frequency_text)

    # Get mean square
    m_square = freq_mean_square_list(frequency_text_, frequency_tags_ai[key], frequency_tags_not_ai[key])

    # Sum the results for a tag
    sum_ai += m_square[0]
    sum_not_ai += m_square[1]


  n = len(tags.items())
  mean_ai = sum_ai/n
  mean_not_ai = sum_not_ai/n

  return mean_ai, mean_not_ai

#### Exemplo de uso de textos separados

In [ ]:
test_text_not_ai = test_not_ai.iat[2, 0]
test_text_ai = test_ai.iat[2, 0]

print("Examplo de uso\n")

print("Texto 1 de teste: humano\n")
print(test_text_not_ai)
mean_ai, mean_not_ai = classify_by_tag_frequency(test_text_not_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

print("\nTexto 1 de teste: IA\n")
print(test_text_ai)
mean_ai, mean_not_ai = classify_by_tag_frequency(test_text_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

Examplo de uso

Texto 1 de teste: humano

Dear State Senator,

The Electoral College has been an election process in the United States sense the constitution was created. But it has come to the time when it no longer has a valuable position and is beginning to deteriorate. Over 60% of voters would prefer to only have a popular vote for Presidential elections, rather than the Electoral College. Recently it seems as if every election has been very tight and the winner of the popular votes loses due to the turn out of the Electoral College votes. The Electoral College system should be abolished and banned from our election process.

In the Electoral College voters vote for a slate of electors who in turn elect the President rather than them directly voting themselves. Source 2 States "The Electors can be anyone not holding office." Therefore, anybody can put their self in the position to be in the Electoral College, even if they are unqualified for the responsibilities it requires. Also, 

#### Contagem de vários textos

In [ ]:
# Case when we want to see how many texts it gets right
correct = 0
wrong = 0

both_texts = pd.concat([test_ai, test_not_ai])
test_size = len(both_texts)

for index, row in both_texts.iterrows():
  text = row['text']
  mean_ai, mean_not_ai = classify_by_tag_frequency(text)
  if mean_ai > mean_not_ai:
    if row['generated'] == 1:
      correct += 1;
    elif row['generated'] == 0:
      wrong += 1
  elif mean_ai < mean_not_ai:
    if row['generated'] == 0:
      correct += 1;
    elif row['generated'] == 1:
      wrong += 1

print(f"Classifications with {test_size} texts")
print(f"Correct: {(correct/test_size) * 100}%\nWrong: {(wrong/test_size) * 100}%")

Classifications with 2000 texts
Correct: 64.4%
Wrong: 35.6%


## **2. Uso uniforme de sinais de pontuação**


Nesse critério, é considerado a frequência de utilização de cada tipo de pontuação para os textos do dataset. Nesse sentido o processo para realizar a classificação através desse método consiste nos seguintes passos:

1. Contagem de frequências de pontuações para textos de IA (média n textos)
2. Contagem de frequências de pontuações para textos de Humano (média n textos)
3. Contagem de frequências de pontuações para o texto a ser classificado
4. Cálculo do erro médio quadrático

**Exemplo:**

1.   Contagem de frequências de pontuações para textos de IA

  * '.': 44.145001189149944
  * '?': 0.7898459112017925
  * '!': 0.9801099024897045
  * ',': 51.21230707606804
  * '-': 2.588591670943434
  * '_': 0.10264241635268936
  * ';': 0.18150183379438972

2.   Contagem de frequências de pontuações para textos de Humano

  * '.': 54.09717094038859
  * '?': 2.068956736523134
  * '!': 0.5703954572076086
  * ',': 40.84438898933055
  * '-': 1.0911359527386622
  * '_': 0.7117211173639582
  * ';': 0.6162308064475058

3.  Contagem de frequências de pontuações para o texto a ser classificado

    * '.': 24
    * '?': 0
    * '!': 0
    * ',': 15
    * '-': 0
    * '_': 0
    * ';': 0

4.   Cálculo do erro médio quadrático

    Ele compara os valores resultante do texto classificado com as duas frequências dos outros textos segundo seu autor.

    * Semelhança com IA: 0.478065806909715

    * Semelhança com Humano: 0.5219341930902851

In [ ]:
# Get the frequency of each punctuation
def punctuation_freq(text):
  dot = 0
  question = 0
  exclamation = 0
  dot_comma = 0
  comma = 0
  line = 0
  underline = 0
  for char in text:
    if char == ';':
      dot_comma += 1
    if char == '.':
      dot += 1
    if char == ',':
      comma += 1
    if char == '?':
      question += 1
    if char == '!':
      exclamation += 1
    if char == '-':
      line += 1
    if char == '_':
      underline += 1
    sum = dot + question + exclamation + comma
  return {'.':dot,'?':question,'!':exclamation,',':comma,'-':line,'_':underline,';':dot_comma}

In [ ]:
def mean_punctuation_freq(_df):
  # Iterate over each row in the DataFrame
  k = 1
  mean_freq = {}
  for index, row in _df.iterrows():
      text_id = index  # Assuming index represents text IDs
      text = row['text']

      # dict with ponctuation and frequency
      freq = punctuation_freq(text)
      for key, value in freq.items():
        if key in mean_freq:
          mean_freq[key] += value
        else:
          mean_freq[key] = value

      k += 1
  total = 0
  for key, value in mean_freq.items():
    total+=value
  for key, value in mean_freq.items():
    mean_freq[key] = 100*(value/total)
  return mean_freq

In [ ]:
# Get the mean of each dataframe used to compare, AI and Human
ai_pont_freq = mean_punctuation_freq(df_ai)
not_ai_pont_freq = mean_punctuation_freq(df_not_ai)

In [ ]:
# Classify a text by the punctuation frequency mean
def classify_by_punctuation(text):
  freq = punctuation_freq(text)
  return freq_mean_square(freq, ai_pont_freq, not_ai_pont_freq)

#### Testando dois textos

In [ ]:
test_text_not_ai = test_not_ai.iat[2, 0]
test_text_ai = test_ai.iat[2, 0]

print("Examplo de uso\n")

print("Texto 1 de teste: humano\n")
print(test_text_not_ai)
mean_ai, mean_not_ai = classify_by_punctuation(test_text_not_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

print("\nTexto 1 de teste: IA\n")
print(test_text_ai)
mean_ai, mean_not_ai = classify_by_punctuation(test_text_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

Examplo de uso

Texto 1 de teste: humano

Dear State Senator,

The Electoral College has been an election process in the United States sense the constitution was created. But it has come to the time when it no longer has a valuable position and is beginning to deteriorate. Over 60% of voters would prefer to only have a popular vote for Presidential elections, rather than the Electoral College. Recently it seems as if every election has been very tight and the winner of the popular votes loses due to the turn out of the Electoral College votes. The Electoral College system should be abolished and banned from our election process.

In the Electoral College voters vote for a slate of electors who in turn elect the President rather than them directly voting themselves. Source 2 States "The Electors can be anyone not holding office." Therefore, anybody can put their self in the position to be in the Electoral College, even if they are unqualified for the responsibilities it requires. Also, 

#### Teste com n textos diversos e classificando como correto e incorreto

In [ ]:
# Case when we want to see how many texts it gets right
correct = 0
wrong = 0

both_texts = pd.concat([test_ai, test_not_ai])
test_size = len(both_texts)

for index, row in both_texts.iterrows():
  text = row['text']
  mean_ai, mean_not_ai = classify_by_punctuation(text)
  if mean_ai > mean_not_ai:
    if row['generated'] == 1:
      correct += 1;
    elif row['generated'] == 0:
      wrong += 1
  elif mean_ai < mean_not_ai:
    if row['generated'] == 0:
      correct += 1;
    elif row['generated'] == 1:
      wrong += 1

print(f"Classifications(%) with {test_size} texts")
print(f"Correct: {(correct/test_size) * 100}%\nWrong: {(wrong/test_size) * 100}%")

Classifications(%) with 2000 texts
Correct: 70.35%
Wrong: 29.65%


## **3. Comprimento das palavras**

Nesse critério, é considerado o comprimento de todas as palavras do texto, ou seja, ele tenta comparar qual o tamanho das palavras mais usadas nos feitos por IA e os feitos por Humanos.

1.   Tokenizar o textos.
2.   Contagem de frequência de cada tamanho para textos de IA.
3.   Contagem de frequência de cada tamanho para textos de Humanos.
4.   Contagem de frequência de cada tamanho do texto a ser classificado.
5.   Comparação entre os resultados com o cálculo do erro médio quadrático.

No caso de textos de IA espera-se que haja uma uniformidade no tamanho das palavras utilizadas, enquanto nos textos de humanos, esse padrão não é comum.

In [ ]:
# Gets the frequency of a certain item of a array and returns the dict of it
def frequency_dictionary(arr):
  frequency_dict = {}
  for item in arr:
    if item in frequency_dict:
      frequency_dict[item] += 1
    else:
      frequency_dict[item] = 1

  return frequency_dict

# Calculates the mean of word lengths
def average_word_length(words):
    # Calculates length of each word
    word_lengths = [len(word) for word in words]

    # Calculates the mean length of words
    average = sum(word_lengths) / len(word_lengths)
    return average

# Calculates and returns a dict of all frequencies by word
def freq_of_words(words):
    # Calculates length of each word
    word_lengths = [len(word) for word in words]

    freq = frequency_dictionary(word_lengths) # key: comprimento | value: frequencia
    ordered_freq = dict(sorted(freq.items(), key=lambda item: item[1]))
    return ordered_freq

In [ ]:
def mean_word_freq(_df):
  # Iterate over each row in the DataFrame
  k = 1
  mean_freq = {}
  word_total = 0
  df_size = len(_df)

  for index, row in _df.iterrows():
      text_id = index  # Assuming index represents text IDs
      text = row['text']

      # Tokenize the text into words
      words = tokenize_phrase(text)
      comprimento_medio = average_word_length(words)
      freq = freq_of_words(words) # key: length | value: frequency
      word_total += len(words)

      for key, value in freq.items():
        if key in mean_freq:
          mean_freq[key] += value
        else:
          mean_freq[key] = value

      k +=1

  for key, value in mean_freq.items():
    mean_freq[key] = value/df_size
  return mean_freq

In [ ]:
ai_word_freq = dict(sorted(mean_word_freq(df_ai).items(), key=lambda item: item[1]))
not_ai_word_freq = dict(sorted(mean_word_freq(df_not_ai).items(), key=lambda item: item[1]))

In [ ]:
def classify_by_word_len(text):
  # Tokenize
  words = tokenize_phrase(text)
  freq = freq_of_words(words) # get frequency

  return freq_mean_square(freq, ai_word_freq, not_ai_word_freq)

#### Teste com dois textos

In [ ]:
test_text_not_ai = test_not_ai.iat[2, 0]
test_text_ai = test_ai.iat[2, 0]

print("Examplo de uso\n")
print("Texto 1 de teste: humano")
mean_ai, mean_not_ai = classify_by_word_len(test_text_not_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

print("\nTexto 1 de teste: IA")
mean_ai, mean_not_ai = classify_by_word_len(test_text_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

Examplo de uso

Texto 1 de teste: humano

Semelhança com IA: 0.32782504376208277 
Semelhança com Humano: 0.6721749562379172


Texto 1 de teste: IA

Semelhança com IA: 0.7597423509171437 
Semelhança com Humano: 0.2402576490828563



#### Teste com n textos diversos e classificando como correto e incorreto

In [ ]:
# Case when we want to see how many texts it gets right
correct = 0
wrong = 0

both_texts = pd.concat([test_ai, test_not_ai])
test_size = len(both_texts)

for index, row in both_texts.iterrows():
  text = row['text']
  mean_ai, mean_not_ai = classify_by_word_len(text)
  if mean_ai > mean_not_ai:
    if row['generated'] == 1:
      correct += 1;
    elif row['generated'] == 0:
      wrong += 1
  elif mean_ai < mean_not_ai:
    if row['generated'] == 0:
      correct += 1;
    elif row['generated'] == 1:
      wrong += 1

print(f"Classifications(%) with {test_size} texts")
print(f"Correct: {(correct/test_size) * 100}%\nWrong: {(wrong/test_size) * 100}%")

Classifications(%) with 2000 texts
Correct: 67.35%
Wrong: 32.65%


## **4. Desvio gramatical**

Nesse critério espera-se avaliar quantas palavras digitadas no texto são escritas da maneira incorreta. No caso dos textos de IA epera-se que haja menos desvio da norma padrão. Apesar desse ponto, é inevitável que haja erros no processo e palavras corretas sejam classificadas como incorretas ou o contrário. Os passos a serem seguidos nessa abordagem são:


1.   Tokenizar textos.
2.   Identificar quantas palavras foram escritas incorretamente nos textos de IA.
3.   Identificar quantas palavras foram escritas incorretamente nos textos de Humanos.
4.   Identificar quantas palavras foram escritas incorretamente no texto alvo.
5.   Comparação dos valores com cálculo do erro médio quadrático.


In [ ]:
spell = SpellChecker()
# Calculates the mean mispelled words in the texts of a dataframe
def mean_misspelled_words(_df):

  mean_freq = {}
  misspelled_total = 0
  k = 0
  for index, row in _df.iterrows():
      text_id = index  # Assuming index represents text IDs
      text = row['text']
      words = tokenize_phrase(text)
      misspelled_count = len(spell.unknown(words))
      misspelled_total += misspelled_count
      k += 1
  return misspelled_total/k


In [ ]:
mispelled_not_ai = mean_misspelled_words(df_not_ai)
mispelled_ai = mean_misspelled_words(df_ai)

In [ ]:
def classify_by_mispelled_freq(text):
  # Tokenize
  words = tokenize_phrase(text)
  misspelled_count = len(spell.unknown(words))

  # Calculates the difference between the values
  mean_ai = abs(misspelled_count - mispelled_ai)
  mean_not_ai = abs(misspelled_count - mispelled_not_ai)

  total = mean_ai+mean_not_ai

  return (1-mean_ai/total, 1-mean_not_ai/total)

#### Teste para dois valores

In [ ]:
test_text_not_ai = test_not_ai.iat[2, 0]
test_text_ai = test_ai.iat[2, 0]

print("Examplo de uso\n")
print("Texto 1 de teste: humano")
mean_ai, mean_not_ai = classify_by_mispelled_freq(test_text_not_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

print("\nTexto 1 de teste: IA")
mean_ai, mean_not_ai = classify_by_mispelled_freq(test_text_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

Examplo de uso

Texto 1 de teste: humano

Semelhança com IA: 0.2984963340375295 
Semelhança com Humano: 0.7015036659624705


Texto 1 de teste: IA

Semelhança com IA: 0.5812659750413471 
Semelhança com Humano: 0.41873402495865286



#### Teste com n textos diversos e classificando como correto e incorreto

In [ ]:
# Case when we want to see how many texts it gets right
correct = 0
wrong = 0

both_texts = pd.concat([test_ai, test_not_ai])
test_size = len(both_texts)

for index, row in both_texts.iterrows():
  text = row['text']
  mean_ai, mean_not_ai = classify_by_mispelled_freq(text)
  if mean_ai > mean_not_ai:
    if row['generated'] == 1:
      correct += 1;
    elif row['generated'] == 0:
      wrong += 1
  elif mean_ai < mean_not_ai:
    if row['generated'] == 0:
      correct += 1;
    elif row['generated'] == 1:
      wrong += 1

print(f"Classifications(%) with {test_size} texts")
print(f"Correct: {((correct/test_size) * 100):.2f}%\nWrong: {((wrong/test_size) * 100):.2f}%")

Classifications(%) with 2000 texts
Correct: 58.30%
Wrong: 41.70%


# **5. Resultado final**

Por fim, vamos unificar todos os critérios utilizados e obter um valor único baseado na média das classificações dos outros procedimentos. Dessa forma, é possível identificar a validade da abordagem produzida, mesmo que seu valor não seja completamente satisfatório.

In [ ]:
# Using all criteria it classifies a text
def classify_all_criteria(text):
  res_ai = 0
  res_not_ai = 0

  criteria_1 = classify_by_tag_frequency(text)
  criteria_2 = classify_by_punctuation(text)
  criteria_3 = classify_by_word_len(text)
  criteria_4 = classify_by_mispelled_freq(text)

  for i,item in enumerate([criteria_1, criteria_2, criteria_3, criteria_4]):
    res_ai += item[0]
    res_not_ai += item[1]

  total = res_ai + res_not_ai
  return (res_ai/total, res_not_ai/total)

#### Classificando dois textos

In [310]:
test_text_not_ai = test_not_ai.iat[2, 0]
test_text_ai = test_ai.iat[2, 0]

print("Examplo de uso\n")
print("Texto 1 de teste: humano")
mean_ai, mean_not_ai = classify_all_criteria(test_text_not_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

print("\nTexto 1 de teste: IA")
mean_ai, mean_not_ai = classify_all_criteria(test_text_ai)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")

Examplo de uso

Texto 1 de teste: humano

Semelhança com IA: 0.3472916110399141 
Semelhança com Humano: 0.6527083889600859


Texto 1 de teste: IA

Semelhança com IA: 0.5795907021039095 
Semelhança com Humano: 0.42040929789609044



In [ ]:
# Case when we want to see how many texts it gets right
correct = 0
wrong = 0

both_texts = pd.concat([test_ai, test_not_ai])
test_size = len(both_texts)

for index, row in both_texts.iterrows():
  text = row['text']
  mean_ai, mean_not_ai = classify_all_criteria(text)
  if mean_ai > mean_not_ai:
    if row['generated'] == 1:
      correct += 1;
    elif row['generated'] == 0:
      wrong += 1
  elif mean_ai < mean_not_ai:
    if row['generated'] == 0:
      correct += 1;
    elif row['generated'] == 1:
      wrong += 1

print(f"Classifications with {test_size} texts")
print(f"Correct: {((correct/test_size) * 100):.2f}%\nWrong: {((wrong/test_size) * 100):.2f}%")

Classifications(%) with 2000 texts
Correct: 67.40%
Wrong: 32.60%


### Teste com um texto qualquer

In [312]:
text = input('Insira um texto para classificação: ')

mean_ai, mean_not_ai = classify_all_criteria(text)
print(f"\nSemelhança com IA: {mean_ai} \nSemelhança com Humano: {mean_not_ai}\n")
if mean_ai > mean_not_ai:
  print("Seu texto foi escrito por uma Inteligência Artificial")
elif mean_ai < mean_not_ai:
  print("Seu texto foi escrito por um humano")
else:
  print("Hmm estamos indecisos, o sistema não soube identificar o autor")


Insira um texto para classificação: The Electoral College has been an election process in the United States sense the constitution was created. But it has come to the time when it no longer has a valuable position and is beginning to deteriorate. Over 60% of voters would prefer to only have a popular vote for Presidential elections, rather than the Electoral College. Recently it seems as if every election has been very tight and the winner of the popular votes loses due to the turn out of the Electoral College votes.

Semelhança com IA: 0.5881341166983055 
Semelhança com Humano: 0.4118658833016945

Seu texto foi escrito por uma Inteligência Artificial
